# Phân Tích Cảm Xúc Comment với Vietnamese-Sentiment-visobert

Notebook này sẽ:
1. Lấy dữ liệu comment từ bảng `nessie.silver_tables.comment` trong khoảng thời gian nhất định
2. Áp dụng model Vietnamese-Sentiment-visobert để phân tích cảm xúc
3. Tổng hợp các chỉ số POS (tích cực), NEG (tiêu cực), NEU (trung lập)

## 1. Import Libraries và Khởi tạo Spark Session

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, to_date, avg, sum as spark_sum
from pyspark.sql.types import *
import os
from datetime import datetime
import pandas as pd

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder.appName("Apply_Sentiment_Model")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "2g")
    .config("spark.executor.cores", "2")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO (S3-compatible) =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    # Propagate environment variables to executors
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    # ===== JAR files =====
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("✅ Spark Session đã được khởi tạo!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")

✅ Spark Session đã được khởi tạo!
Spark Master: spark://spark-master:7077
Application ID: app-20251208151458-0001


## 2. Load Vietnamese Sentiment Model

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Đường dẫn đến model
model_path = "/opt/spark-apps/Vietnamese-Sentiment-visobert"

print("⏳ Đang tải model Vietnamese-Sentiment-visobert...")

# Load tokenizer và model (use_fast=False để tránh lỗi tokenizer.json)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Kiểm tra và sử dụng GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

print(f"✅ Model đã được tải thành công!")
print(f"Device: {device}")
print(f"Labels: {model.config.id2label}")

⏳ Đang tải model Vietnamese-Sentiment-visobert...
✅ Model đã được tải thành công!
Device: cpu
Labels: {0: 'NEG', 1: 'POS', 2: 'NEU'}


## 3. Định nghĩa Hàm Dự Đoán Cảm Xúc

In [9]:
def predict_sentiment(text):
    """
    Dự đoán cảm xúc của văn bản
    Returns: sentiment label (POS, NEG, NEU)
    """
    if not text or text.strip() == "":
        return "NEU"
    
    try:
        # Tokenize
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Dự đoán
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = F.softmax(logits, dim=-1)
            predicted_class = torch.argmax(probabilities, dim=-1).item()
        
        # Chuyển đổi id sang label
        sentiment = model.config.id2label[predicted_class]
        return sentiment
    except Exception as e:
        print(f"Error processing text: {str(e)}")
        return "NEU"

# Test function
test_text = "Trường này rất tốt, giảng viên nhiệt tình!"
print(f"Test text: {test_text}")
print(f"Sentiment: {predict_sentiment(test_text)}")

Test text: Trường này rất tốt, giảng viên nhiệt tình!
Sentiment: POS


## 4. Lấy Dữ Liệu Comment từ Bảng Silver (Batch Processing)

In [15]:
# Cấu hình batch processing
BATCH_SIZE = 20000  # Số lượng comment xử lý mỗi batch

print("🔍 Đang load batch comment chưa xử lý từ Silver...")

# ===== TỐI ƯU: Dùng SQL với LEFT ANTI JOIN (SUBQUERY) =====
# Cách này NHANH NHẤT vì:
# 1. Spark optimize toàn bộ query thành 1 execution plan
# 2. Iceberg pruning: chỉ scan partition/file cần thiết
# 3. KHÔNG load toàn bộ Silver hay Gold vào memory
# 4. Push-down predicates xuống storage layer

try:
    # Query với LEFT ANTI JOIN ngay trong SQL
    # Spark sẽ optimize và chỉ scan cần thiết
    df_comments = spark.sql(f"""
        SELECT 
            s.articleID as postID,
            s.commentID,
            s.comment,
            s.commentTime,
            s.commentLike,
            s.levelComment,
            s.numberOfReply,
            s.created_at,
            s.updated_at
        FROM nessie.silver_tables.comment s
        LEFT ANTI JOIN nessie.gold_result_model_multi_task.Comment_Sentiment g
            ON s.commentID = g.commentID
        WHERE s.comment IS NOT NULL
          AND TRIM(s.comment) != ''
        LIMIT {BATCH_SIZE}
    """)
    
    total_comments = df_comments.count()
    
    # # Đếm số comment đã xử lý (nếu cần thống kê)
    # try:
    #     existing_count = spark.sql("""
    #         SELECT COUNT(DISTINCT commentID) as count
    #         FROM nessie.gold_result_model_multi_task.Comment_Sentiment
    #     """).collect()[0]['count']
    #     print(f"✅ Đã có trong Gold: {existing_count:,} comment")
    # except:
    #     print("✅ Bảng Gold đang trống")
    
    print(f"📦 Batch hiện tại: {total_comments:,} comment")
    
except Exception as e:
    print(f"⚠️ Bảng Gold chưa tồn tại: {str(e)}")
    print("📝 Load batch đầu tiên từ Silver...")
    
    # Nếu Gold chưa có, lấy batch đầu tiên
    df_comments = spark.sql(f"""
        SELECT 
            articleID as postID,
            commentID,
            comment,
            commentTime,
            commentLike,
            levelComment,
            numberOfReply,
            created_at,
            updated_at
        FROM nessie.silver_tables.comment
        WHERE comment IS NOT NULL
          AND TRIM(comment) != ''
        LIMIT {BATCH_SIZE}
    """)
    
    total_comments = df_comments.count()
    print(f"📦 Batch đầu tiên: {total_comments:,} comment")

if total_comments > 0:
    print(f"\n✅ Sẵn sàng xử lý {total_comments:,} comment")
else:
    print("\n🎉 Tất cả comment đã được xử lý!")
    print("💡 Không có comment mới cần phân tích.")

🔍 Đang load batch comment chưa xử lý từ Silver...


[Stage 69:===================================================>(6667 + 4) / 6672]

📦 Batch hiện tại: 20,000 comment

✅ Sẵn sàng xử lý 20,000 comment


## 5. Áp Dụng Model Sentiment cho Comments

In [16]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, TimestampType

import pandas as pd
from datetime import datetime

# ===== HÀM XỬ LÝ GIÁ TRỊ TIMESTAMP =====
def safe_ts(v):
    """Chuyển pandas NaT / NaN / None thành None hoặc datetime."""
    if v is None or pd.isna(v):
        return None
    if isinstance(v, pd.Timestamp):
        return v.to_pydatetime()
    return v

print("⏳ Đang áp dụng model sentiment...")

if total_comments > 0:
    print("\n📥 Bước 1: Thu thập dữ liệu về driver...")
    comments_pd = df_comments.toPandas()
    print(f"✅ Thu thập {len(comments_pd)} comment")

    batch_save_size = 1000
    results = []

    schema = StructType([
        StructField("postID", StringType(), True),
        StructField("commentID", LongType(), True),
        StructField("comment", StringType(), True),
        StructField("sentiment", StringType(), True),
        StructField("commentTime", TimestampType(), True),
        StructField("commentLike", IntegerType(), True),
        StructField("levelComment", IntegerType(), True),
        StructField("numberOfReply", IntegerType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("updated_at", TimestampType(), True)
    ])

    print("\n📋 Bước 2: Chuẩn bị bảng Gold...")
    spark.sql("CREATE DATABASE IF NOT EXISTS nessie.gold_result_model_multi_task")

    spark.sql("""
    CREATE TABLE IF NOT EXISTS nessie.gold_result_model_multi_task.Comment_Sentiment (
        postID STRING,
        commentID BIGINT,
        comment STRING,
        sentiment STRING,
        commentTime TIMESTAMP,
        commentLike INT,
        levelComment INT,
        numberOfReply INT,
        created_at TIMESTAMP,
        updated_at TIMESTAMP
    ) USING iceberg
    """)

    print("✅ Bảng đã tồn tại hoặc được tạo mới.")

    print(f"\n🔄 Bước 3: Xử lý {len(comments_pd)} comment...")

    for idx, row in comments_pd.iterrows():

        # ---- Predict sentiment ----
        sentiment = predict_sentiment(row['comment'])

        # ---- Append kết quả vào list ----
        results.append({
            'postID': row['postID'],
            'commentID': int(row['commentID']) if not pd.isna(row['commentID']) else None,
            'comment': row['comment'],
            'sentiment': sentiment,
            'commentTime': safe_ts(row['commentTime']),
            'commentLike': int(row['commentLike']) if not pd.isna(row['commentLike']) else 0,
            'levelComment': int(row['levelComment']) if not pd.isna(row['levelComment']) else 0,
            'numberOfReply': int(row['numberOfReply']) if not pd.isna(row['numberOfReply']) else 0,
            'created_at': safe_ts(row['created_at']),
            'updated_at': datetime.now()
        })

        if (idx + 1) % 500 == 0:
            print(f"   ✓ Đã xử lý {idx+1}/{len(comments_pd)} comment...")

        # ---- Lưu batch ----
        if len(results) >= batch_save_size:
            print(f"\n💾 Lưu batch {len(results)} vào Gold...")
            df_batch = spark.createDataFrame(results, schema=schema)
            df_batch.writeTo("nessie.gold_result_model_multi_task.Comment_Sentiment") \
                .using("iceberg") \
                .append()
            print("   → Đã lưu xong batch")
            results = []

    # ---- Lưu batch cuối ----
    if len(results) > 0:
        print(f"\n💾 Lưu batch cuối {len(results)} comment...")
        df_batch = spark.createDataFrame(results, schema=schema)
        df_batch.writeTo("nessie.gold_result_model_multi_task.Comment_Sentiment") \
            .using("iceberg") \
            .append()
        print("   → Đã lưu xong batch cuối")

    print("\n🎉 Hoàn tất phân tích sentiment và lưu Gold layer!")

    print("\n🔍 Ví dụ 10 dòng mới nhất từ Gold:")
    df_gold_sample = spark.sql("""
        SELECT postID, commentID, comment, sentiment, commentTime, commentLike
        FROM nessie.gold_result_model_multi_task.Comment_Sentiment
        ORDER BY updated_at DESC
        LIMIT 10
    """)
    df_gold_sample.show(10, truncate=50)

    df_with_sentiment = spark.sql("""
        SELECT * FROM nessie.gold_result_model_multi_task.Comment_Sentiment
        ORDER BY updated_at DESC
        LIMIT {}
    """.format(total_comments))

else:
    print("⚠️ Không có comment để xử lý!")


⏳ Đang áp dụng model sentiment...

📥 Bước 1: Thu thập dữ liệu về driver...


✅ Thu thập 20000 comment

📋 Bước 2: Chuẩn bị bảng Gold...
✅ Bảng đã tồn tại hoặc được tạo mới.

🔄 Bước 3: Xử lý 20000 comment...
   ✓ Đã xử lý 500/20000 comment...
   ✓ Đã xử lý 1000/20000 comment...

💾 Lưu batch 1000 vào Gold...
   → Đã lưu xong batch
   ✓ Đã xử lý 1500/20000 comment...
   ✓ Đã xử lý 2000/20000 comment...

💾 Lưu batch 1000 vào Gold...
   → Đã lưu xong batch
   ✓ Đã xử lý 2500/20000 comment...
   ✓ Đã xử lý 3000/20000 comment...

💾 Lưu batch 1000 vào Gold...
   → Đã lưu xong batch
   ✓ Đã xử lý 3500/20000 comment...
   ✓ Đã xử lý 4000/20000 comment...

💾 Lưu batch 1000 vào Gold...
   → Đã lưu xong batch
   ✓ Đã xử lý 4500/20000 comment...
   ✓ Đã xử lý 5000/20000 comment...

💾 Lưu batch 1000 vào Gold...
   → Đã lưu xong batch
   ✓ Đã xử lý 5500/20000 comment...
   ✓ Đã xử lý 6000/20000 comment...

💾 Lưu batch 1000 vào Gold...
   → Đã lưu xong batch
   ✓ Đã xử lý 6500/20000 comment...
   ✓ Đã xử lý 7000/20000 comment...

💾 Lưu batch 1000 vào Gold...
   → Đã lưu xong bat

[Stage 100:====================================================>(208 + 4) / 212]

+----------------------------------------------+---------+-----------------------------------------+---------+-------------------+-----------+
|                                        postID|commentID|                                  comment|sentiment|        commentTime|commentLike|
+----------------------------------------------+---------+-----------------------------------------+---------+-------------------+-----------+
|         @khoibeooto/video/7494220834781809928|    59337|                        Em muốn học sao ạ|      POS|2025-05-11 00:00:00|          1|
|    @cepdgnlcand.cep/video/7458558566757059847|    59458|                              Có bạn nha!|      NEU|2025-01-23 00:00:00|          0|
|@vococenter_official/video/6976630345588280578|    59285|                   đc. nếu e thích & muốn|      POS|2021-08-08 00:00:00|          0|
|         @luatsubinh/video/6988687885616696602|    59852|              đại học luật Huế ổn không ạ|      NEU|2021-08-22 00:00:00|          2|

## 6. Tổng Hợp Các Chỉ Số Sentiment (POS, NEG, NEU)

In [20]:
# if total_comments > 0:
#     print("📊 THỐNG KÊ TỔNG QUAN")
#     print("=" * 60)
    
#     # 1. Đếm số lượng theo từng loại sentiment
#     sentiment_counts = df_with_sentiment.groupBy("sentiment").count().orderBy("count", ascending=False)
    
#     print("\n1️⃣ Phân bố Sentiment:")
#     sentiment_counts.show()
    
#     # 2. Tính phần trăm
#     sentiment_stats = sentiment_counts.withColumn(
#         "percentage",
#         (col("count") / total_comments * 100)
#     )
    
#     print("\n2️⃣ Phân bố Sentiment (%)")
#     sentiment_stats.show()
    
#     # 3. Tổng hợp chi tiết
#     pos_count = df_with_sentiment.filter(col("sentiment") == "POS").count()
#     neg_count = df_with_sentiment.filter(col("sentiment") == "NEG").count()
#     neu_count = df_with_sentiment.filter(col("sentiment") == "NEU").count()
    
#     pos_pct = (pos_count / total_comments * 100) if total_comments > 0 else 0
#     neg_pct = (neg_count / total_comments * 100) if total_comments > 0 else 0
#     neu_pct = (neu_count / total_comments * 100) if total_comments > 0 else 0
    
#     print("\n3️⃣ Tổng Hợp Chi Tiết:")
#     print(f"   🟢 POSITIVE (POS): {pos_count:,} comment ({pos_pct:.2f}%)")
#     print(f"   🔴 NEGATIVE (NEG): {neg_count:,} comment ({neg_pct:.2f}%)")
#     print(f"   ⚪ NEUTRAL (NEU):  {neu_count:,} comment ({neu_pct:.2f}%)")
#     print(f"   📊 TỔNG CỘNG:      {total_comments:,} comment")
    
# else:
#     print("⚠️ Không có dữ liệu để tổng hợp!")

## 7. Phân Tích Sentiment Theo Thời Gian

In [21]:
# if total_comments > 0:
#     # Phân tích theo ngày
#     df_by_date = df_with_sentiment.withColumn("date", to_date(col("commentTime")))
    
#     sentiment_by_date = df_by_date.groupBy("date", "sentiment").count() \
#         .orderBy("date", ascending=False)
    
#     print("📅 Phân bố Sentiment theo Ngày:")
#     sentiment_by_date.show(20)
    
#     # Pivot để dễ nhìn hơn
#     sentiment_pivot = df_by_date.groupBy("date").pivot("sentiment").count() \
#         .fillna(0) \
#         .orderBy("date", ascending=False)
    
#     print("\n📊 Bảng Pivot Sentiment theo Ngày:")
#     sentiment_pivot.show(20)
# else:
#     print("⚠️ Không có dữ liệu để phân tích theo thời gian!")

## 8. Phân Tích Sentiment Theo Bài Viết (Article)

In [22]:
# if total_comments > 0:
#     # Top bài viết có nhiều comment tích cực nhất
#     pos_by_article = df_with_sentiment.filter(col("sentiment") == "POS") \
#         .groupBy("postID") \
#         .agg(count("*").alias("positive_count")) \
#         .orderBy("positive_count", ascending=False)
    
#     print("🟢 Top 10 Bài Viết có nhiều Comment TÍCH CỰC nhất:")
#     pos_by_article.show(10, truncate=False)
    
#     # Top bài viết có nhiều comment tiêu cực nhất
#     neg_by_article = df_with_sentiment.filter(col("sentiment") == "NEG") \
#         .groupBy("postID") \
#         .agg(count("*").alias("negative_count")) \
#         .orderBy("negative_count", ascending=False)
    
#     print("\n🔴 Top 10 Bài Viết có nhiều Comment TIÊU CỰC nhất:")
#     neg_by_article.show(10, truncate=False)
    
#     # Tổng hợp sentiment cho từng article
#     sentiment_by_article = df_with_sentiment.groupBy("postID").pivot("sentiment").count() \
#         .fillna(0) \
#         .withColumn("total_comments", col("POS") + col("NEG") + col("NEU")) \
#         .orderBy("total_comments", ascending=False)
    
#     print("\n📊 Tổng Hợp Sentiment theo Bài Viết (Top 20):")
#     sentiment_by_article.show(20, truncate=False)
# else:
#     print("⚠️ Không có dữ liệu để phân tích theo bài viết!")

## 9. Visualization - Trực Quan Hóa Kết Quả

In [23]:
# if total_comments > 0:
#     import matplotlib.pyplot as plt
#     import seaborn as sns
    
#     # Set style
#     sns.set_style("whitegrid")
    
#     # Chuyển đổi sang Pandas để vẽ biểu đồ
#     sentiment_pd = sentiment_stats.toPandas()
    
#     # Tạo figure với 2 subplots
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
#     # 1. Bar chart
#     colors = {'POS': '#4CAF50', 'NEG': '#F44336', 'NEU': '#9E9E9E'}
#     bar_colors = [colors.get(s, '#9E9E9E') for s in sentiment_pd['sentiment']]
    
#     ax1.bar(sentiment_pd['sentiment'], sentiment_pd['count'], color=bar_colors)
#     ax1.set_xlabel('Sentiment', fontsize=12)
#     ax1.set_ylabel('Số lượng Comment', fontsize=12)
#     ax1.set_title('Phân Bố Sentiment - Bar Chart', fontsize=14, fontweight='bold')
    
#     # Thêm số liệu lên cột
#     for i, (sent, cnt) in enumerate(zip(sentiment_pd['sentiment'], sentiment_pd['count'])):
#         ax1.text(i, cnt, f'{int(cnt):,}', ha='center', va='bottom', fontsize=10)
    
#     # 2. Pie chart
#     pie_colors = [colors.get(s, '#9E9E9E') for s in sentiment_pd['sentiment']]
    
#     ax2.pie(sentiment_pd['percentage'], 
#             labels=sentiment_pd['sentiment'], 
#             autopct='%1.1f%%',
#             colors=pie_colors,
#             startangle=90,
#             textprops={'fontsize': 11})
#     ax2.set_title('Phân Bố Sentiment - Pie Chart', fontsize=14, fontweight='bold')
    
#     plt.tight_layout()
#     plt.show()
    
#     print("\n✅ Đã tạo biểu đồ trực quan!")
# else:
#     print("⚠️ Không có dữ liệu để vẽ biểu đồ!")

## 10. Lưu Kết Quả vào Gold Layer

In [24]:
# from pyspark.sql.functions import current_timestamp

# # Tự động lưu kết quả vào bảng Gold layer
# save_results = True  # Tự động lưu sau khi phân tích

# if save_results and total_comments > 0:
#     print("💾 Đang lưu kết quả vào bảng Gold...")
    
#     # Tạo database gold nếu chưa có
#     spark.sql("CREATE DATABASE IF NOT EXISTS nessie.gold_result_model_multi_task")
    
#     # Tạo bảng Comment_Sentiment nếu chưa có
#     spark.sql("""
#     CREATE TABLE IF NOT EXISTS nessie.gold_result_model_multi_task.Comment_Sentiment (
#         commentID BIGINT,
#         postID STRING,
#         comment STRING,
#         sentiment STRING,
#         commentTime TIMESTAMP,
#         commentLike INT,
#         levelComment INT,
#         numberOfReply INT,
#         created_at TIMESTAMP,
#         updated_at TIMESTAMP
#     ) USING iceberg
#     TBLPROPERTIES (
#         'write.format.default' = 'parquet',
#         'write.metadata.compression-codec' = 'gzip'
#     )
#     """)
    
#     print("✅ Đã tạo/kiểm tra bảng: nessie.gold_result_model_multi_task.Comment_Sentiment")
    
#     # Chuẩn bị dữ liệu để lưu
#     df_to_save = df_with_sentiment.select(
#         col("postID"),
#         col("commentID"),
#         col("comment"),
#         col("sentiment"),
#         col("commentTime"),
#         col("commentLike"),
#         col("levelComment"),
#         col("numberOfReply"),
#         col("created_at"),
#         current_timestamp().alias("updated_at")  # Cập nhật thời gian xử lý
#     )
    
#     # Lưu kết quả vào bảng Gold
#     print(f"📤 Đang lưu {total_comments:,} bản ghi vào bảng Gold...")
    
#     df_to_save.writeTo("nessie.gold_result_model_multi_task.Comment_Sentiment") \
#         .using("iceberg") \
#         .tableProperty("write.format.default", "parquet") \
#         .tableProperty("write.metadata.compression-codec", "gzip") \
#         .append()
    
#     print("✅ Đã lưu kết quả sentiment vào bảng: nessie.gold_result_model_multi_task.Comment_Sentiment")
    
#     # Kiểm tra số lượng bản ghi đã lưu
#     saved_count = spark.sql("SELECT COUNT(*) as count FROM nessie.gold_result_model_multi_task.Comment_Sentiment").collect()[0]['count']
#     print(f"📊 Tổng số bản ghi trong bảng Gold: {saved_count:,}")
    
# elif total_comments > 0:
#     print("ℹ️ Bỏ qua việc lưu kết quả. Đặt save_results=True nếu muốn lưu.")
# else:
#     print("⚠️ Không có dữ liệu để lưu!")

## 11. Kiểm Tra Dữ Liệu Đã Lưu trong Gold Layer

In [25]:
# if save_results and total_comments > 0:
#     print("\n" + "=" * 60)
#     print("🔍 KIỂM TRA DỮ LIỆU TRONG GOLD LAYER")
#     print("=" * 60)
    
#     # Đọc dữ liệu từ bảng Gold
#     df_gold = spark.sql("SELECT * FROM nessie.gold_result_model_multi_task.Comment_Sentiment")
    
#     print("\n📊 Thống kê tổng quan:")
#     total_in_gold = df_gold.count()
#     print(f"   Tổng số bản ghi: {total_in_gold:,}")
    
#     # Phân bố sentiment trong Gold
#     print("\n📈 Phân bố Sentiment trong Gold Layer:")
#     df_gold.groupBy("sentiment").count().orderBy("count", ascending=False).show()
    
#     # Hiển thị mẫu dữ liệu
#     print("\n📝 Mẫu dữ liệu từ Gold Layer (10 bản ghi mới nhất):")
#     df_gold.select("postID", "commentID", "comment", "sentiment", "commentTime", "commentLike", "updated_at") \
#         .orderBy("updated_at", ascending=False) \
#         .show(10, truncate=50)
    
#     # Thống kê tiến độ
#     print(f"\n📊 Tiến độ xử lý:")
#     print(f"   - Đã xử lý: {total_in_gold:,} comment")
#     print(f"   - Còn lại: {max(0, total_silver_comments - total_in_gold):,} comment")
#     progress_pct = (total_in_gold / total_silver_comments * 100) if total_silver_comments > 0 else 0
#     print(f"   - Tiến độ: {progress_pct:.2f}%")
    
#     print("\n✅ Kiểm tra hoàn tất!")
# else:
#     print("\nℹ️ Bỏ qua kiểm tra do không lưu dữ liệu.")

## 12. Tổng Kết

In [26]:
# print("\n" + "=" * 60)
# print("📋 TÓM TẮT KẾT QUẢ PHÂN TÍCH SENTIMENT")
# print("=" * 60)

# if total_comments > 0:
#     print(f"\n📦 Batch Processing:")
#     print(f"   - Tổng comment trong Silver: {total_silver_comments:,}")
#     print(f"   - Batch hiện tại: {total_comments:,} comment")
#     print(f"   - Kích thước batch: {BATCH_SIZE:,} comment/batch")
    
#     print(f"\n📊 Kết quả phân tích batch này:")
#     print(f"   🟬 Comment TÍCH CỰC (POS): {pos_count:,} ({pos_pct:.2f}%)")
#     print(f"   🔴 Comment TIÊU CỰC (NEG): {neg_count:,} ({neg_pct:.2f}%)")
#     print(f"   ⚪ Comment TRUNG LẬP (NEU): {neu_count:,} ({neu_pct:.2f}%)")
    
#     # Đánh giá tổng quan
#     print("\n📈 Đánh giá batch hiện tại:")
#     if pos_pct > 50:
#         print("   ✅ Tâm lý cộng đồng chủ yếu TÍCH CỰC")
#     elif neg_pct > 50:
#         print("   ⚠️ Tâm lý cộng đồng chủ yếu TIÊU CỰC")
#     else:
#         print("   ℹ️ Tâm lý cộng đồng TRUNG LẬP hoặc TRỘN LẪN")
    
#     # Tính toán số comment còn lại
#     try:
#         processed_total = existing_comment_ids + total_comments
#     except:
#         processed_total = total_comments
    
#     remaining = total_silver_comments - processed_total
    
#     if remaining > 0:
#         print(f"\n🔄 Tiến độ:")
#         print(f"   - Đã xử lý: {processed_total:,} comment")
#         print(f"   - Còn lại: {remaining:,} comment")
#         progress_pct = (processed_total / total_silver_comments * 100) if total_silver_comments > 0 else 0
#         print(f"   - Hoàn thành: {progress_pct:.2f}%")
#         print(f"\n💡 Chạy lại notebook để xử lý batch tiếp theo")
#     else:
#         print("\n✅ Đã xử lý hết tất cả comment mới!")
        
# else:
#     print("\n✅ Tất cả comment đã được xử lý!")
#     if total_silver_comments > 0:
#         print(f"📊 Tổng số comment trong Silver: {total_silver_comments:,}")
#     print("💡 Không có comment mới cần phân tích.")

# print("\n" + "=" * 60)
# print("✅ Hoàn thành phân tích!")
# print("=" * 60)

## 13. Dừng Spark Session

In [27]:
# Giải phóng cache
if total_comments > 0:
    df_with_sentiment.unpersist()

# Dừng Spark Session
spark.stop()
print("🛑 Spark Session đã được dừng!")
print("✅ Hoàn tất!")

🛑 Spark Session đã được dừng!
✅ Hoàn tất!
